In [2]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
sns.set_palette("Blues_d")

from IPython.display import HTML

# import the widgets module
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

warnings.filterwarnings("ignore")
#sns.set(style="whitegrid")

In [3]:
def make_count(count, number):
    return np.array([np.ones(count) * number])

def make_voter_array(counts):
    bases = [-2, -1, 0, 1, 2]
    arr = np.array([])
    for i, j in zip(counts, bases):
        arr = np.append(arr, make_count(i, j))
    return np.array([int(i) for i in arr])

In [4]:
politicalStances = {-2: 'Extreme Left', -1:'Moderate Left', 0:'Centrist',
                   1:'Moderate Right', 2:'Extreme Right'}
colors = {-2:'blue', -1:'blue', 0:'black', 1:'red', 2:'red'}

def make_color_arr(dta):
    arr = []
    for i in dta:
        arr.append(colors[i])
    return arr

In [5]:
def plot_voters(counts):
    pos = [] 
    keys = {} # this dict will help to keep track ...
    
    data = make_voter_array(counts)

    # this loop will give us a list of frequencies to each number
    for num in data: 
        if num not in keys:
            keys[num] = 1
            pos.append(1)
        else:
            keys[num] += 1
            pos.append(keys[num])
    
    
    for key, value in zip(keys.keys(), keys.values()):
        print('There are ' + str(value) + ' ' + politicalStances[key] + 
              ' (' + str(key) + ') ' + 'voters')
        
    colorArr = make_color_arr(data)
    
    plt.scatter(data, pos, c=colorArr)
    plt.grid(False)
    plt.xticks([-2, -1, 0, 1, 2])
    plt.show()

In [13]:
def plot_voters_candidates(counts, candA, candB):
    
    pos = [] 
    keys = {} # this dict will help to keep track ...
    
    data = make_voter_array(counts)
    voting = {}
    candidates = {'candA': 'Candidate A', 'candB': 'Candidate B'}

    # this loop will give us a list of frequencies to each number
    for num in data: 
        if num not in keys:
            keys[num] = 1
            pos.append(1)
        else:
            keys[num] += 1
            pos.append(keys[num])
    
    voter_pos = set(data)
    for i in voter_pos:
        if i == 0:
            voting[i] = np.random.choice(['candA', 'candB'], 
                                         p=[0.5, 0.5])
            continue
        diffA = abs(i - candA)
        diffB = abs(i - candB)
        if diffA < diffB:
            voting[i] = 'candA'
        elif diffB == diffA:
            #distributing the voters 
            voting[i] = ('candA', 'candB')        
        else:
            voting[i] = 'candB'
        

    for key, value in zip(keys.keys(), keys.values()):
        who_voting = voting[i]
        half = False
        if type(who_voting) == tuple:   
            half = True 
        if half == False: 
            print('There are ' + str(value) + ' ' + politicalStances[key] + 
                  ' (' + str(key) + ') ' + 'voters who will vote for ' + candidates[who_voting])
        else:
            half = False
            int_value = value // 2
            if int_value == 0:
                who_voting_now = np.random.choice(['Candidate A','Candidate B'], p=[.5, .5])
                print('There are ' + str(value) + ' ' + politicalStances[key] + 
                      ' (' + str(key) + ') ' + 'voters who will vote for ' + who_voting_now)
                continue
            if half == False:
                who_voting_now = who_voting[0]
                print('There are ' + str(int_value) + ' ' + politicalStances[key] + 
                  ' (' + str(key) + ') ' + 'voters who will vote for ' + candidates[who_voting_now])
                half = True
            if half == True:
                who_voting_now = who_voting[1]
                print('There are ' + str(int_value) + ' ' + politicalStances[key] + 
                  ' (' + str(key) + ') ' + 'voters who will vote for ' + candidates[who_voting_now])

                

    
        
    colorArr = make_color_arr(data)
    
            
    plt.scatter(data, pos, c=colorArr)
    plt.grid(False)
    plt.xticks([-2, -1, 0, 1, 2])
    
    plt.scatter([candA, candB], [0, 0], c=['gold', 'lime'])
    plt.annotate('Candidate A', (candA, 0), (candA, 20), arrowprops=dict(arrowstyle='wedge', 
                                                                   connectionstyle='arc3, rad=0.2'))
    plt.annotate('Candidate B', (candB, 0), (candB, 10), arrowprops=dict(arrowstyle='wedge', 
                                                                   connectionstyle='arc3, rad=-0.2'))
    
    
    
    plt.show()

# 2. Median Voter Theorem (MVT) 

#### Finding the Median

NumPy allows us to make useful calculations with numerical data. Imagine we have some array of numbers -- our data -- and that we wish to find the median value of the data. A NumPy method that allows us to do this is `np.median` (where `np` refers to NumPy), which takes as input some data and outputs the median value for that data. Run the cell below, which should generate 72 random numbers.

In [14]:
data = np.random.randint(0,100,size=72)
data

array([78, 41, 25,  3,  7, 21, 90,  1, 53, 40,  7, 79, 88, 33, 72, 28, 25,
       72, 68, 23, 84, 59, 27, 75, 60, 94, 60, 69,  3, 63, 10, 84, 30, 88,
       80,  7, 54, 41, 99, 33, 21, 47, 96, 77, 68, 50, 15, 73, 48, 60, 62,
       51, 79, 86, 54, 70, 64,  9, 22, 24, 74, 29, 43, 47, 95, 99, 30, 57,
       17, 79, 54, 87])

<font color="blue"> <b> Now, with the `data` above, use the `np.median` method to find the median value for the data. (1 pt) </font> </b> 

In [15]:
median = np.median(data)
median

54.0

Here is a quote about campaign strategy from a Liberal activist: “The key data is this, and it’s
important to reemphasize if only to shut up the useless, overpaid political consultants who
idiotically babble about ‘moving to the center’ or ‘compromising with the other side’...What
matters is turning out our voters. That’s it. The Democrats win when we fire up and turn out our
base.” (A party’s “base” is considered to be its most ideologically committed voters. For
example, the Pizza4All party’s base is pizza fanatics.)  


Why might it be better to pick a platform to please the base rather than the median voter? Let’s
try a variant of the Downs model from class. Suppose there are five possible voter ideal points: 
 
| Number | Political Stance       |
|--------|------------------------|
| -2     | Extreme or "Base" Left |
| -1     | Moderate Left          |
| 0      | Centrist               |
| 1      | Moderate Right         |
| 2      | Extreme Right          |

Other than this, retain the same assumptions of the model from class: there
are two parties A and B, who don’t care about policy and prefer winning to tying to losing. The
parties simultaneously set platforms, and the voters vote for whichever party proposes a closer
platform.

**Part 1:** A naive way to predict that parties will cater to the bases is if there are lots of
extreme voters and few centrists. Run the cells below to generate 101 voters and their political stances.
Imagine there is 45 extreme left voters, 5 moderate left, 1 centrist, 5 moderate right, and 45 extreme right voters. 

Run the cell below to generate a list of voters.

In [19]:
voters = make_voter_array([45, 5, 1, 5, 45])
voters

array([-2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2,
       -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2,
       -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -1, -1, -1, -1, -1,  0,
        1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2])

<font color="blue"> <b> Using `np.median`, compute the median of the voter array. (1 pt) </font> </b> 

In [20]:
median = np.median(voters)
median

0.0

<font color="blue"> <b> Use the Median Voter Theorem (MVT) to show that there is a unique Nash Equilibrium to this
game where both parties propose the platform of the single centrist voter. (2 pt) </font> </b> 

In [21]:
interact(plot_voters_candidates, counts=fixed([45, 5, 1, 5, 45]), 
         candA=widgets.IntSlider(min=-2, max=2, step=1, value=0), 
         candB=widgets.IntSlider(min=-2, max=2, step=1, value=0));

interactive(children=(IntSlider(value=0, description='candA', max=2, min=-2), IntSlider(value=0, description='…

**Part 2:** Now, suppose we remove the centrist voter. Since the remaining electorate has an even number 100 voters, we cannot directly use the MVT derived in class (which needs an odd number).  

In [213]:
voters = make_voter_array([45, 5, 0, 5, 45])
voters

array([-2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2,
       -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2,
       -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -1, -1, -1, -1, -1,  1,
        1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2])

<font color="blue"> <b> Using the sliders below, configure the base each candidate caters to. Argue why there is still no Nash Equillibrium where a party proposes the platform preferred by a base voter (-2 or 2). No need to do any math, just show logically why at least one party could change their strategy and increase their payoff.  (2 pt) </font> </b> 

In [23]:
interact(plot_voters_candidates, counts=fixed([45, 5, 0, 5, 45]), 
         candA=widgets.IntSlider(min=-2, max=2, step=1, value=-2), 
         candB=widgets.IntSlider(min=-2, max=2, step=1, value=2));

interactive(children=(IntSlider(value=-2, description='candA', max=2, min=-2), IntSlider(value=2, description=…

**Part 3** 

<font color="blue"> <b> Argue why, in the 100 person electorate, there is an NE where one party offers the platform -1 and the other offers platform 1.  (3 pt) </font> </b> 

In [24]:
interact(plot_voters_candidates, counts=fixed([45, 5, 1, 5, 45]), 
         candA=widgets.IntSlider(min=-2, max=2, step=1, value=-2), 
         candB=widgets.IntSlider(min=-2, max=2, step=1, value=2));

interactive(children=(IntSlider(value=-2, description='candA', max=2, min=-2), IntSlider(value=2, description=…

**Part 4** In the cell below, write in the number of voters for each base, with the first item being the number of extreme left, the second being moderate left, the third centrists, the fourth moderate right, the fifth extreme right. 

<font color="blue"> <b> Modify the number of voters in the `int` variable to get a NE where parties propose the platforms preferred by the extreme voters, their base (-2 or 2).   (3 pt) </font> </b>

In [ ]:
# [number of extreme left, number of moderate left, number centrists, number moderate right, number extreme right]
inter = [.. , .. , .. , .. , .. ]
voters = make_voter_array(inter)

In [ ]:
interact(plot_voters_candidates, counts=fixed(inter), 
         candA=widgets.IntSlider(min=-2, max=2, step=1, value=0), 
         candB=widgets.IntSlider(min=-2, max=2, step=1, value=0));